# EU Funds - Data cleaning

Data came from the official EU Fund database of Hungary: https://www.palyazat.gov.hu/tamogatott_projektkereso

Relevant Development plans:
* EU 2007-2013 (Funds between 2007 and 2013)
* NFT (Funds before 2007)
* Széchenyi 2020 (Funds between 2014-2020)
* KTIA (Research and Development Funds) ?

In [268]:
import os
import pandas as pd
import numpy as np
import unidecode

In [208]:
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list

In [209]:
os.chdir('C:/Users/User/Documents/Thesis/Data/raw_data/funds')

In [210]:
NFT_tender = pd.read_excel('NFT.xlsx', 'Pályázat')
SZ2020_tender = pd.read_excel('Széchenyi 2020.xlsx', 'Pályázat')
EU0713_tender = pd.read_excel('EU 2007-2013.xlsx', 'Pályázat')
KTIA_tender = pd.read_excel('KTIA.xlsx', 'Pályázat')

In [211]:
tender = pd.concat([NFT_tender,SZ2020_tender, EU0713_tender,KTIA_tender])

In [212]:
tender.shape #ok, number of rows is same

(140901, 17)

In [213]:
#tender without duplicates
tender_wd = tender.drop_duplicates(subset=['Pályázat ID'], keep='last')
tender_wd.shape

(140901, 17)

In [214]:
NFT_contract = pd.read_excel('NFT.xlsx', 'Szerződés')
SZ2020_contract = pd.read_excel('Széchenyi 2020.xlsx', 'Szerződés')
EU0713_contract = pd.read_excel('EU 2007-2013.xlsx', 'Szerződés')
KTIA_contract = pd.read_excel('KTIA.xlsx', 'Szerződés')

In [215]:
contract = pd.concat([NFT_contract, SZ2020_contract, EU0713_contract, KTIA_contract])

In [216]:
contract.shape

(140901, 39)

In [217]:
#contract without duplicates
contract_wd = contract.drop_duplicates(subset=['Pályázat ID'], keep='last')
contract_wd.shape #ok no duplicates

(140901, 39)

In [218]:
#merge contact and tender data
data_merged = tender.merge(contract, on='Pályázat ID')
data_merged.shape

(140901, 55)

In [219]:
#merged data without duplicates
data_merged_wd = data_merged.drop_duplicates(subset=['Pályázat ID'], keep='last')
data_merged_wd.shape #ok no duplicates

(140901, 55)

### Data Cleaning

In [220]:
#Dropping rows with 0 or missing funds (they might cancelled the agreement before signing the contract)
data_merged_wd = data_merged_wd[data_merged_wd['Megitelt Támogatás Num'] != 0]
data_merged_wd = data_merged_wd[data_merged_wd['Megitelt Támogatás Num'].notna()]
data_merged_wd.shape #139 696 rows remained

(139696, 55)

In [221]:
#number of missing values per columns
data_merged_wd.isna().sum()

Pályázat ID                       0
Fejlesztési program név           0
Forrás_x                       2728
Kiirás éve                   131069
op kod                            0
Konstrukció név                   0
Konstrukció kód                   0
Pályázó neve                      1
Projekt címe                      1
Projekt helyszíne (régió)      1601
Megval megye név               1601
Járás név                      1630
Kistérség név                  1677
Helység név                    1648
Támogatási döntés dátuma          0
Megitélt támogatás                0
Load date                         0
ID                                0
Projekt Címe                      1
Palyazo Neve                      1
Szervezet Neve               139696
Regió                          7098
Megye                          7098
Helység                        7145
Konstrukció Név                   0
Beavatkozási Kategória       104655
Tipus Megnevezés             139696
Op Név                      

In [222]:
data_merged_wd = data_merged_wd[['Pályázat ID',                      
'Fejlesztési program név',          
'op kod',                           
'Konstrukció név',                 
'Konstrukció kód',                  
'Pályázó neve',                     
'Projekt címe',                    
'Projekt helyszíne (régió)',    
'Megval megye név',           
'Járás név',                  
'Helység név',           
'Támogatási döntés dátuma',          
'Megitélt támogatás',               
'Projekt Címe',                      
'Palyazo Neve',                     
'Konstrukció Név',                   
'Op Név',                           
'Forrás_y',                        
'Típus',                             
'Támogatás Arány',                   
'Projekt Megval Kezdete',          
'Projekt Megval Vége',             
'Tám Dönt Dátum',                  
'Fejl Kezd Tényleges',           
'Szerződött Összeg',               
'Projekt Összköltsége',          
'Kifiz Elő',                       
'Megitélt Támogatás',    
'Megitelt Támogatás Num',            
'Utolsó Kifiz Dat',                
'Hatalyba Lép Datum',           
'Kifiz Szla',                      
'Fejl bef tényleges']]     

data_merged1

In [45]:
# create a choice list from the programs
#it is manually examined in excel to decide health-relevant programs
choices = data_merged1['Konstrukció Név'].values.tolist()
choices
a = Remove(choices)
a = pd.DataFrame(a) 
a.to_csv('construction_name.csv')

In [46]:
# create a choice list from the beneficiaries
choices = data_merged1['Pályázó neve'].values.tolist()
choices
Remove(choices)
b = Remove(choices)
b = pd.DataFrame(b) 
b.to_csv('beneficiaries_name.csv')

In [250]:
program_filters =[
'HEFOP-4.3',
'HEFOP-4.3.1',
'HEFOP-4.4',
'EFOP-1.10.1-VEKOP-16',
'EFOP-1.10.2-17',
'EFOP-1.10.3-17',
'EFOP-1.10.4-18',
'EFOP-1.12.1-17',
'EFOP-1.8.0-VEKOP-17',
'EFOP-1.8.1-VEKOP-15',
'EFOP-1.8.19-17',
'EFOP-1.8.2-17',
'EFOP-1.8.20-17',
'EFOP-1.8.21-18',
'EFOP-1.9.6-16',
'EFOP-2.2.0-16',
'EFOP-2.2.1-VEKOP-16',
'EFOP-2.2.18-17',
'EFOP-2.2.19-17',
'EFOP-2.2.20-17',
'EFOP-2.2.6-VEKOP-16',
'EFOP-3.8.1-14',
'KEHOP-5.2.1-15',
'TOP-4.1.1-15-BO1',
'TOP-6.6.1-15-SL1',
'TOP-6.6.1-16-VP1',
'VEKOP-6.3.5-17',
'VEKOP-7.2.2-17',
'VEKOP-7.2.3-17',
'VEKOP-7.2.4-17',
'DAOP-4.1.1/A',
'DAOP-4.1.1/A-09',
'DAOP-4.1.1/B',
'DAOP-4.1.1/B-09',
'DAOP-4.1.1/C-09',
'DAOP-4.1.1/C-10',
'DAOP-4.1.2/B-11',
'DAOP-5.1.3-11',
'DDOP-3.1.3/A-2f',
'DDOP-3.1.3/B',
'DDOP-3.1.3/C-11',
'DDOP-3.1.3/G-14',
'EKOP-2.3.7-2012',
'EKOP-3.1.1-09',
'GOP-1.3.1-11/F',
'KDOP-5.2.1/A',
'KDOP-5.2.1/A-09',
'KDOP-5.2.1/B',
'KDOP-5.2.1/B-09',
'KDOP-5.2.1/C-11',
'KEOP-5.6.0/E/15',
'KEOP-7.4.0.',
'KMOP-1.1.4-11/F',
'KMOP-4.3.1/A',
'KMOP-4.3.1/A_2-09-2f',
'KMOP-4.3.1/B-11',
'KMOP-4.3.1/C',
'KMOP-4.3.1/C_2-09-2f',
'KMOP-4.3.2',
'KMOP-4.3.2/A-13',
'KMOP-4.3.2/B-13',
'KMOP-4.3.3.A-12',
'KMOP-4.3.3/B-09-2F',
'KMOP-4.3.3/B_2',
'NYDOP-5.2.1/A',
'NYDOP-5.2.1/A-09',
'NYDOP-5.2.1/A-12',
'NYDOP-5.2.1/B',
'NYDOP-5.2.1/C',
'NYDOP-5.2.1/C-11',
'TIOP-2.1.2-07/1',
'TIOP-2.1.2-08/1',
'TIOP-2.1.3-07/1',
'TIOP-2.1.3-08/1',
'TIOP-2.1.3-10/1',
'TIOP-2.2.1-11/1',
'TIOP-2.2.2-08/2',
'TIOP-2.2.2/C-10/1',
'TIOP-2.2.3-11/1',
'TIOP-2.2.4-09/1',
'TIOP-2.2.5-09/1',
'TIOP-2.2.6-12/1A',
'TIOP-2.2.6-12/1B',
'TIOP-2.2.7-07/2F/2',
'TIOP-2.2.9-15/1',
'TIOP-2.3.4-09/2',
'TÁMOP-4.1.1.C-13/1/KONV',
'TÁMOP-4.2.6-15/1',
'TÁMOP-5.5.7-08/1',
'TÁMOP-6.1.1-12/1',
'TÁMOP-6.1.2-11/1',
'TÁMOP-6.1.2-11/2',
'TÁMOP-6.1.2-11/3',
'TÁMOP-6.1.2-11/4',
'TÁMOP-6.1.2-13/1',
'TÁMOP-6.1.2-13/2',
'TÁMOP-6.1.2.A-14/1',
'TÁMOP-6.1.2.A-14/2',
'TÁMOP-6.1.2.B-14/1',
'TÁMOP-6.1.2/A-09/1',
'TÁMOP-6.1.2/A-09/1-KMR',
'TÁMOP-6.1.2/LHH-09/1',
'TÁMOP-6.1.2/LHH-09/2',
'TÁMOP-6.1.2/LHH/11-A',
'TÁMOP-6.1.2/LHH/11-B',
'TÁMOP-6.1.3-08/1',
'TÁMOP-6.1.3.A-13/1',
'TÁMOP-6.1.3.B-12/1',
'TÁMOP-6.1.6-14/1',
'TÁMOP-6.1.7-15/1',
'TÁMOP-6.2.1-11/1',
'TÁMOP-6.2.2.A-11/1',
'TÁMOP-6.2.2.A-KMR/11-1',
'TÁMOP-6.2.2.B-12/1',
'TÁMOP-6.2.2/A-09/1',
'TÁMOP-6.2.2/A-09/2',
'TÁMOP-6.2.2/A-KMR-09/1',
'TÁMOP-6.2.2/A-KMR-09/2',
'TÁMOP-6.2.2/B-09/1',
'TÁMOP-6.2.2/B-09/2',
'TÁMOP-6.2.3-12/1',
'TÁMOP-6.2.4.A-11/1',
'TÁMOP-6.2.4.B-12/1',
'TÁMOP-6.2.4.B-12/2',
'TÁMOP-6.2.4.B-14/2',
'TÁMOP-6.2.4/A-08/1',
'TÁMOP-6.2.4/A-08/1/konv',
'TÁMOP-6.2.4/A-09/1',
'TÁMOP-6.2.4/A-09/1/KMR',
'TÁMOP-6.2.5-B-13/1',
'TÁMOP-6.2.5.A-12/1',
'TÁMOP-6.2.7-13/1',
'TÁMOP-6.2.8-15/1',
'ÉAOP-4.1.2/A',
'ÉAOP-4.1.2/A-09',
'ÉAOP-4.1.2/A-12',
'ÉAOP-4.1.2/B',
'ÉAOP-4.1.2/B-09',
'ÉAOP-4.1.2/C-11',
'ÉAOP-4.1.2/D-09',
'ÉAOP-4.1.2/D-10',
'ÉMOP-4.1.1/A',
'ÉMOP-4.1.1/A-09',
'ÉMOP-4.1.1/A-12',
'ÉMOP-4.1.1/B',
'ÉMOP-4.1.1/B-09',
'ÉMOP-4.1.1/B-12',
'ÉMOP-4.1.1/C-09',
'ÉMOP-4.1.1/C-10',
'ÉMOP-4.1.2/A-11',
'KTIA_13-2.3.3'
'NKFP_06_A1',
'NKFP_07_A1']

In [251]:
hrelated2 = data_merged_wd.loc[(data_merged_wd['Konstrukció kód'].str.contains('|'.join(program_filters), case=True)),]
hrelated2.shape

(3978, 33)

In [230]:
#hrelated2.to_csv('health_realated_2.csv')

In [256]:
type_key = pd.read_excel('type_codes.xlsx', 'Sheet3')

In [257]:
#health_funds = hrelated2.merge(type_key, how='left',on='Konstrukció kód')
health_funds = hrelated2.merge(type_key,on='Konstrukció kód')
health_funds.shape

(3978, 34)

In [258]:
#health_funds.tail(50)

,Pályázat ID,Fejlesztési program név,op kod,Konstrukció név,Konstrukció kód,Pályázó neve,Projekt címe,Projekt helyszíne (régió),Megval megye név,Járás név,...,Szerződött Összeg,Projekt Összköltsége,Kifiz Elő,Megitélt Támogatás,Megitelt Támogatás Num,Utolsó Kifiz Dat,Hatalyba Lép Datum,Kifiz Szla,Fejl bef tényleges,type
3928,11459802,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SIROK KÖZSÉGI ÖNKORMÁNYZAT,A háziorvosi- és a fogorvosi rendelő felújítás...,Észak-Magyarország,Heves,Pétervásárai,...,4.259740e+07,NaN,6.569128e+06,4.259740e+07,4.259740e+07,2015-09-21 01:00:00,2014-06-16 01:00:00,3.376421e+07,2015-07-08T23:00:00.000Z,primary_outpatient
3929,11737602,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SOMOSKŐÚJFALU KÖZSÉG ÖNKORMÁNYZATA,Fogászati-védőnői rendelő felújítása Somoskőúj...,Észak-Magyarország,Nógrád,Salgótarjáni,...,3.556449e+07,NaN,3.576238e+06,3.556449e+07,3.556449e+07,2015-07-22 01:00:00,2014-06-16 01:00:00,3.167775e+07,2015-11-17T23:00:00.000Z,primary_outpatient
3930,11741602,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZAJLA KÖZSÉGI ÖNKORMÁNYZAT,Szajla község orvosi rendelőinek komplex felúj...,Észak-Magyarország,Heves,Pétervásárai,...,3.802655e+07,NaN,3.654913e+06,3.822256e+07,3.822256e+07,2015-07-06 01:00:00,2014-06-16 01:00:00,3.410567e+07,2015-11-18T23:00:00.000Z,primary_outpatient
3931,11847902,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZALASZEND KÖZSÉG ÖNKORMÁNYZATA,Egészségügyi alapellátás fejlesztése Szalaszen...,Észak-Magyarország,Borsod-Abaúj-Zemplén,Encsi,...,6.000000e+07,NaN,0.000000e+00,6.000000e+07,6.000000e+07,2015-08-25 01:00:00,2013-08-05 01:00:00,5.979762e+07,2015-11-16T23:00:00.000Z,primary_outpatient
3932,11587902,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZÉCSÉNY VÁROS ÖNKORMÁNYZATA,Orvosi rendelő korszrűsításe a határmenti Pöst...,Észak-Magyarország,Nógrád,Szécsényi,...,1.671570e+07,NaN,4.178924e+06,1.671570e+07,1.671570e+07,2016-05-13 01:00:00,2015-05-15 01:00:00,1.238504e+07,2015-09-29T23:00:00.000Z,primary_outpatient
3933,11682702,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZENTISTVÁN NAGYKÖZSÉG ÖNKORMÁNYZATA,"Háziorvosi, fogorvosi rendelő és védőnői szolg...",Észak-Magyarország,Borsod-Abaúj-Zemplén,Mezőkövesdi,...,5.992480e+07,NaN,8.245112e+06,5.992480e+07,5.992480e+07,2014-11-27 00:00:00,2013-10-16 01:00:00,5.098909e+07,2014-11-26T23:00:00.000Z,primary_outpatient
3934,11693502,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZIHALOM KÖZSÉGI ÖNKORMÁNYZAT,Egészségügyi alapellátás fejlesztés Szihalom k...,Észak-Magyarország,Heves,Füzesabonyi,...,5.804328e+07,NaN,2.383254e+07,5.804328e+07,5.804328e+07,2014-10-30 00:00:00,2013-07-15 01:00:00,3.421074e+07,2014-10-29T23:00:00.000Z,primary_outpatient
3935,11849102,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZIKSZÓ VÁROS ÖNKORMÁNYZATA,Szikszó város egészségügyi alapellátásának egy...,Észak-Magyarország,Borsod-Abaúj-Zemplén,Szikszói,...,5.994996e+07,NaN,5.355225e+06,5.994996e+07,5.994996e+07,2015-07-08 01:00:00,2013-07-25 01:00:00,5.458452e+07,2015-05-27T23:00:00.000Z,primary_outpatient
3936,11753402,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZIN KÖZSÉGI ÖNKORMÁNYZAT,Orvosi rendelő felújítása Szin településen,Észak-Magyarország,Borsod-Abaúj-Zemplén,Edelényi,...,2.647508e+07,NaN,0.000000e+00,2.647508e+07,2.647508e+07,2015-02-27 00:00:00,2014-06-10 01:00:00,2.638713e+07,2015-08-31T23:00:00.000Z,primary_outpatient
3937,11723702,EU 2007-2013,ÉMOP,"Egészségügyi alapellátás, egészségházak és já...",ÉMOP-4.1.1/A-12,SZŰCSI KÖZSÉGI ÖNKORMÁNYZAT,Egészségügyi alapellátás fejlesztése Szűcsi Kö...,Észak-Magyarország,Heves,Gyöngyösi,...,3.630861e+07,NaN,1.533468e+07,3.649576e+07,3.649576e+07,20

In [259]:
health_funds.isna().sum()
#850 data is missing for actual end of project in case of health aweraness and research project going to delete that

Pályázat ID                     0
Fejlesztési program név         0
op kod                          0
Konstrukció név                 0
Konstrukció kód                 0
Pályázó neve                    0
Projekt címe                    0
Projekt helyszíne (régió)       1
Megval megye név                1
Járás név                       1
Helység név                     2
Támogatási döntés dátuma        0
Megitélt támogatás              0
Projekt Címe                    0
Palyazo Neve                    0
Konstrukció Név                 0
Op Név                          0
Forrás_y                        4
Típus                           0
Támogatás Arány                 0
Projekt Megval Kezdete        850
Projekt Megval Vége           850
Tám Dönt Dátum                  0
Fejl Kezd Tényleges          1382
Szerződött Összeg             532
Projekt Összköltsége         3453
Kifiz Elő                      44
Megitélt Támogatás            824
Megitelt Támogatás Num          0
Utolsó Kifiz D

In [260]:
health_funds.to_csv('health_funds.csv')

In [261]:
health_funds = health_funds[health_funds['Projekt Megval Vége'].notna()]
health_funds.shape

(3128, 34)

In [283]:
health_funds.isna().sum()

id                           0
fund_name                    0
op_code                      0
program_name                 0
program_code                 0
applicant_name               0
tender_title                 0
village_name                 2
date_decision                0
amount_awarded               0
source_fund                  4
ratio_support                0
start_of_imlementation       0
end_of_implementation        0
start_actual               532
amount_contracted          519
amount_total              2614
advance_payment             16
amount_awarded               0
amount_awarded2              0
date_last_payment           16
date_effective               0
invoice_payment            225
end_actual                 582
dtype: int64

In [263]:
#merged data without duplicates
health_funds = health_funds.drop_duplicates(subset=['Pályázat ID'], keep='last')
health_funds.shape #ok no duplicates

(3128, 34)

In [265]:
health_funds = health_funds[[
'Pályázat ID',
'Fejlesztési program név',
'op kod',                          
'Konstrukció név',                 
'Konstrukció kód',                 
'Pályázó neve',                    
'Projekt címe',                    
'Helység név',                     
'Támogatási döntés dátuma',        
'Megitélt támogatás',              
'Forrás_y',                        
'Támogatás Arány',                 
'Projekt Megval Kezdete',          
'Projekt Megval Vége',                               
'Fejl Kezd Tényleges',         
'Szerződött Összeg',             
'Projekt Összköltsége',         
'Kifiz Elő',                      
'Megitélt Támogatás',          
'Megitelt Támogatás Num',  
'Utolsó Kifiz Dat',
'Hatalyba Lép Datum', 
'Kifiz Szla', 
'Fejl bef tényleges'
]]     

health_funds

,Pályázat ID,Fejlesztési program név,op kod,Konstrukció név,Konstrukció kód,Pályázó neve,Projekt címe,Helység név,Támogatási döntés dátuma,Megitélt támogatás,...,Fejl Kezd Tényleges,Szerződött Összeg,Projekt Összköltsége,Kifiz Elő,Megitélt Támogatás,Megitelt Támogatás Num,Utolsó Kifiz Dat,Hatalyba Lép Datum,Kifiz Szla,Fejl bef tényleges
15,1209740201,Széchenyi 2020,EFOP,Ápoló tanulók részére pályaválasztást támogató...,EFOP-1.10.1-VEKOP-16,EGÉSZSÉGÜGYI NYILVÁNTARTÁSI ÉS KÉPZÉSI KÖZPONT,Ápoló tanulók részére pályaválasztást támogató...,Budapest,2017-02-07 01:00:00,4.400000e+09,...,NaN,NaN,4.400000e+09,2.157210e+09,4.400000e+09,4.400000e+09,2018-01-02 00:00:00,2020-05-04 16:16:55,6.405081e+07,NaN
16,1542580201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,ALBERT SCHWEITZER KÓRHÁZ-RENDELŐINTÉZET,Egészségügyi humánerőforrás fejlesztés az Albe...,Hatvan,2018-02-07 01:00:00,8.619863e+07,...,NaN,NaN,8.619863e+07,4.633829e+07,8.619863e+07,8.619863e+07,2020-04-28 01:00:00,2020-05-05 12:14:36,3.750264e+07,NaN
17,1602790201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,ALMÁSI BALOGH PÁL KÓRHÁZ,Humánerőforrás fejlesztés az Almási Balogh Pál...,Ózd,2018-01-10 01:00:00,1.497179e+08,...,NaN,NaN,1.497179e+08,7.511878e+07,1.497179e+08,1.497179e+08,2020-06-05 01:00:00,2020-06-03 11:32:04,6.624433e+07,NaN
18,1560420201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,BÁCS-KISKUN MEGYEI KÓRHÁZ A SZEGEDI TUDOMÁNYEG...,Egészségügyi humánerőforrás-fejlesztés,Kecskemét,2017-12-19 01:00:00,1.487405e+08,...,NaN,NaN,1.487405e+08,0.000000e+00,1.487405e+08,1.487405e+08,2019-10-10 01:00:00,2018-11-27 11:10:01,1.461526e+08,NaN
19,1604920201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,BAJAI SZENT RÓKUS KÓRHÁZ,Bajai Szent Rókus Kórház humánerőforrás-fejles...,Baja,2018-01-10 01:00:00,1.494150e+08,...,NaN,NaN,1.494150e+08,5.134631e+07,1.494150e+08,1.494150e+08,2019-06-14 01:00:00,2019-05-08 01:00:00,9.806869e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,5881302,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Ricse Nagyközség Önkormányzata,"Egészségügyi központ nyílászáró cseréje, homlo...",Ricse,2011-06-09 02:00:00,1.712726e+07,...,2011-09-01 01:00:00,1.712726e+07,NaN,4.215049e+06,1.712726e+07,1.712726e+07,2013-06-20 01:00:00,2011-08-08 01:00:00,1.291221e+07,2013-06-09T23:00:00.000Z
3963,5882702,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Sóstófalva Község Önkormányzat,Orvosi rendelő felújítása,Sóstófalva,2011-06-09 02:00:00,1.269458e+07,...,2011-10-05 01:00:00,1.269458e+07,NaN,0.000000e+00,1.269458e+07,1.269458e+07,2013-04-10 01:00:00,2011-08-05 01:00:00,1.262985e+07,2012-09-29T23:00:00.000Z
3964,5879502,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Tiszabábolna Község Önkormányzata,Orvosi rendelők és védőnői szolgálatok felújít...,Tiszabábolna,2011-06-09 02:00:00,2.614222e+07,...,2011-10-17 01:00:00,2.614222e+07,NaN,1.521318e+06,2.614222e+07,2.614222e+07,2013-03-13 00:00:00,2011-10-06 01:00:00,2.461376e+07,2012-09-29T23:00:00.000Z
3965,5882802,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Tiszaladány Községi Önkormányzat,Orvosi rendelő bővítése és a védőnői szolgálat...,Tiszaladány,2011-06-09 02:00:00,1.930096e+07,...,2011-09-01 01:00:00,1.930096e+07,NaN,3.822521e+06,1.930096e+07,1.930096e+07,2013-06-07 01:00:00,2011-09-19 01:00:00,1.512913e+07,2012-12-30T23:00:00.000Z


In [266]:
health_funds = health_funds.rename(columns = {
'Pályázat ID': 'id',
'Fejlesztési program név': 'fund_name',
'op kod': 'op_code',
'Konstrukció név': 'program_name',
'Konstrukció kód': 'program_code',
'Pályázó neve': 'applicant_name',
'Projekt címe': 'tender_title',
'Helység név': 'village_name',
'Támogatási döntés dátuma': 'date_decision',        
'Megitélt támogatás': 'amount_awarded',              
'Forrás_y': "source_fund",                        
'Támogatás Arány': "ratio_support",                 
'Projekt Megval Kezdete': "start_of_imlementation",          
'Projekt Megval Vége': "end_of_implementation",             
'Fejl Kezd Tényleges': "start_actual",         
'Szerződött Összeg': 'amount_contracted',             
'Projekt Összköltsége': "amount_total",         
'Kifiz Elő': "advance_payment",                      
'Megitélt Támogatás': "amount_awarded",          
'Megitelt Támogatás Num': "amount_awarded2",  
'Utolsó Kifiz Dat': "date_last_payment",
'Hatalyba Lép Datum': 'date_effective', 
'Kifiz Szla': 'invoice_payment', 
'Fejl bef tényleges': 'end_actual'},inplace = False)
                 
health_funds                  

,id,fund_name,op_code,program_name,program_code,applicant_name,tender_title,village_name,date_decision,amount_awarded,...,start_actual,amount_contracted,amount_total,advance_payment,amount_awarded,amount_awarded2,date_last_payment,date_effective,invoice_payment,end_actual
15,1209740201,Széchenyi 2020,EFOP,Ápoló tanulók részére pályaválasztást támogató...,EFOP-1.10.1-VEKOP-16,EGÉSZSÉGÜGYI NYILVÁNTARTÁSI ÉS KÉPZÉSI KÖZPONT,Ápoló tanulók részére pályaválasztást támogató...,Budapest,2017-02-07 01:00:00,4.400000e+09,...,NaN,NaN,4.400000e+09,2.157210e+09,4.400000e+09,4.400000e+09,2018-01-02 00:00:00,2020-05-04 16:16:55,6.405081e+07,NaN
16,1542580201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,ALBERT SCHWEITZER KÓRHÁZ-RENDELŐINTÉZET,Egészségügyi humánerőforrás fejlesztés az Albe...,Hatvan,2018-02-07 01:00:00,8.619863e+07,...,NaN,NaN,8.619863e+07,4.633829e+07,8.619863e+07,8.619863e+07,2020-04-28 01:00:00,2020-05-05 12:14:36,3.750264e+07,NaN
17,1602790201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,ALMÁSI BALOGH PÁL KÓRHÁZ,Humánerőforrás fejlesztés az Almási Balogh Pál...,Ózd,2018-01-10 01:00:00,1.497179e+08,...,NaN,NaN,1.497179e+08,7.511878e+07,1.497179e+08,1.497179e+08,2020-06-05 01:00:00,2020-06-03 11:32:04,6.624433e+07,NaN
18,1560420201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,BÁCS-KISKUN MEGYEI KÓRHÁZ A SZEGEDI TUDOMÁNYEG...,Egészségügyi humánerőforrás-fejlesztés,Kecskemét,2017-12-19 01:00:00,1.487405e+08,...,NaN,NaN,1.487405e+08,0.000000e+00,1.487405e+08,1.487405e+08,2019-10-10 01:00:00,2018-11-27 11:10:01,1.461526e+08,NaN
19,1604920201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,BAJAI SZENT RÓKUS KÓRHÁZ,Bajai Szent Rókus Kórház humánerőforrás-fejles...,Baja,2018-01-10 01:00:00,1.494150e+08,...,NaN,NaN,1.494150e+08,5.134631e+07,1.494150e+08,1.494150e+08,2019-06-14 01:00:00,2019-05-08 01:00:00,9.806869e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,5881302,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Ricse Nagyközség Önkormányzata,"Egészségügyi központ nyílászáró cseréje, homlo...",Ricse,2011-06-09 02:00:00,1.712726e+07,...,2011-09-01 01:00:00,1.712726e+07,NaN,4.215049e+06,1.712726e+07,1.712726e+07,2013-06-20 01:00:00,2011-08-08 01:00:00,1.291221e+07,2013-06-09T23:00:00.000Z
3963,5882702,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Sóstófalva Község Önkormányzat,Orvosi rendelő felújítása,Sóstófalva,2011-06-09 02:00:00,1.269458e+07,...,2011-10-05 01:00:00,1.269458e+07,NaN,0.000000e+00,1.269458e+07,1.269458e+07,2013-04-10 01:00:00,2011-08-05 01:00:00,1.262985e+07,2012-09-29T23:00:00.000Z
3964,5879502,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Tiszabábolna Község Önkormányzata,Orvosi rendelők és védőnői szolgálatok felújít...,Tiszabábolna,2011-06-09 02:00:00,2.614222e+07,...,2011-10-17 01:00:00,2.614222e+07,NaN,1.521318e+06,2.614222e+07,2.614222e+07,2013-03-13 00:00:00,2011-10-06 01:00:00,2.461376e+07,2012-09-29T23:00:00.000Z
3965,5882802,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,Tiszaladány Községi Önkormányzat,Orvosi rendelő bővítése és a védőnői szolgálat...,Tiszaladány,2011-06-09 02:00:00,1.930096e+07,...,2011-09-01 01:00:00,1.930096e+07,NaN,3.822521e+06,1.930096e+07,1.930096e+07,2013-06-07 01:00:00,2011-09-19 01:00:00,1.512913e+07,2012-12-30T23:00:00.000Z


In [287]:
health_funds[health_funds['village_name'].isnull()]
health_funds.loc[health_funds['program_code'] == 'TÁMOP-6.1.1-12/1', 'village_name'] = "national"
health_funds.loc[health_funds['program_code'] == 'TÁMOP-6.1.2/LHH-09/1', 'village_name'] = "tiszafured"

In [288]:
health_funds.loc[health_funds['village_name'] == 'Kömlő', 'village_name'] = "komlo2"
health_funds.loc[health_funds['village_name'] == 'Kömörő', 'village_name'] = "komoro2"

health_funds['village_name'] = health_funds['village_name'].str.lower()
health_funds['village_name'] = health_funds['village_name'].apply(unidecode.unidecode)
health_funds['village_name'] = health_funds['village_name'].str.strip()

health_funds['applicant_name'] = health_funds['applicant_name'].str.lower()
health_funds['applicant_name'] = health_funds['applicant_name'].apply(unidecode.unidecode)
health_funds['applicant_name'] = health_funds['applicant_name'].str.strip()

health_funds

,id,fund_name,op_code,program_name,program_code,applicant_name,tender_title,village_name,date_decision,amount_awarded,...,start_actual,amount_contracted,amount_total,advance_payment,amount_awarded,amount_awarded2,date_last_payment,date_effective,invoice_payment,end_actual
15,1209740201,Széchenyi 2020,EFOP,Ápoló tanulók részére pályaválasztást támogató...,EFOP-1.10.1-VEKOP-16,egeszsegugyi nyilvantartasi es kepzesi kozpont,Ápoló tanulók részére pályaválasztást támogató...,budapest,2017-02-07 01:00:00,4.400000e+09,...,NaN,NaN,4.400000e+09,2.157210e+09,4.400000e+09,4.400000e+09,2018-01-02 00:00:00,2020-05-04 16:16:55,6.405081e+07,NaN
16,1542580201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,albert schweitzer korhaz-rendelointezet,Egészségügyi humánerőforrás fejlesztés az Albe...,hatvan,2018-02-07 01:00:00,8.619863e+07,...,NaN,NaN,8.619863e+07,4.633829e+07,8.619863e+07,8.619863e+07,2020-04-28 01:00:00,2020-05-05 12:14:36,3.750264e+07,NaN
17,1602790201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,almasi balogh pal korhaz,Humánerőforrás fejlesztés az Almási Balogh Pál...,ozd,2018-01-10 01:00:00,1.497179e+08,...,NaN,NaN,1.497179e+08,7.511878e+07,1.497179e+08,1.497179e+08,2020-06-05 01:00:00,2020-06-03 11:32:04,6.624433e+07,NaN
18,1560420201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,bacs-kiskun megyei korhaz a szegedi tudomanyeg...,Egészségügyi humánerőforrás-fejlesztés,kecskemet,2017-12-19 01:00:00,1.487405e+08,...,NaN,NaN,1.487405e+08,0.000000e+00,1.487405e+08,1.487405e+08,2019-10-10 01:00:00,2018-11-27 11:10:01,1.461526e+08,NaN
19,1604920201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,bajai szent rokus korhaz,Bajai Szent Rókus Kórház humánerőforrás-fejles...,baja,2018-01-10 01:00:00,1.494150e+08,...,NaN,NaN,1.494150e+08,5.134631e+07,1.494150e+08,1.494150e+08,2019-06-14 01:00:00,2019-05-08 01:00:00,9.806869e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,5881302,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,ricse nagykozseg onkormanyzata,"Egészségügyi központ nyílászáró cseréje, homlo...",ricse,2011-06-09 02:00:00,1.712726e+07,...,2011-09-01 01:00:00,1.712726e+07,NaN,4.215049e+06,1.712726e+07,1.712726e+07,2013-06-20 01:00:00,2011-08-08 01:00:00,1.291221e+07,2013-06-09T23:00:00.000Z
3963,5882702,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,sostofalva kozseg onkormanyzat,Orvosi rendelő felújítása,sostofalva,2011-06-09 02:00:00,1.269458e+07,...,2011-10-05 01:00:00,1.269458e+07,NaN,0.000000e+00,1.269458e+07,1.269458e+07,2013-04-10 01:00:00,2011-08-05 01:00:00,1.262985e+07,2012-09-29T23:00:00.000Z
3964,5879502,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,tiszababolna kozseg onkormanyzata,Orvosi rendelők és védőnői szolgálatok felújít...,tiszababolna,2011-06-09 02:00:00,2.614222e+07,...,2011-10-17 01:00:00,2.614222e+07,NaN,1.521318e+06,2.614222e+07,2.614222e+07,2013-03-13 00:00:00,2011-10-06 01:00:00,2.461376e+07,2012-09-29T23:00:00.000Z
3965,5882802,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,tiszaladany kozsegi onkormanyzat,Orvosi rendelő bővítése és a védőnői szolgálat...,tiszaladany,2011-06-09 02:00:00,1.930096e+07,...,2011-09-01 01:00:00,1.930096e+07,NaN,3.822521e+06,1.930096e+07,1.930096e+07,2013-06-07 01:00:00,2011-09-19 01:00:00,1.512913e+07,2012-12-30T23:00:00.000Z


In [310]:
#extracting years
health_funds['date_effective'] = pd.to_datetime(health_funds['date_effective'],format='%Y-%m-%d-%H:%M')
health_funds['date_last_payment'] = pd.to_datetime(health_funds['date_last_payment'],format='%y-%m-%d%H:%M')
health_funds['year_effective'] = pd.DatetimeIndex(health_funds['date_effective']).year
health_funds['year_lp'] = pd.DatetimeIndex(health_funds['date_last_payment']).year.astype('Int64')
#df['month'] = pd.DatetimeIndex(df['date']).month

In [311]:
health_funds

,id,fund_name,op_code,program_name,program_code,applicant_name,tender_title,village_name,date_decision,amount_awarded,...,advance_payment,amount_awarded,amount_awarded2,date_last_payment,date_effective,invoice_payment,end_actual,year_effective,year_last_payment,year_lp
15,1209740201,Széchenyi 2020,EFOP,Ápoló tanulók részére pályaválasztást támogató...,EFOP-1.10.1-VEKOP-16,egeszsegugyi nyilvantartasi es kepzesi kozpont,Ápoló tanulók részére pályaválasztást támogató...,budapest,2017-02-07 01:00:00,4.400000e+09,...,2.157210e+09,4.400000e+09,4.400000e+09,2018-01-02 00:00:00,2020-05-04 16:16:55,6.405081e+07,NaN,2020,2018-01-02 00:00:00,2018
16,1542580201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,albert schweitzer korhaz-rendelointezet,Egészségügyi humánerőforrás fejlesztés az Albe...,hatvan,2018-02-07 01:00:00,8.619863e+07,...,4.633829e+07,8.619863e+07,8.619863e+07,2020-04-28 01:00:00,2020-05-05 12:14:36,3.750264e+07,NaN,2020,2020-04-28 01:00:00,2020
17,1602790201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,almasi balogh pal korhaz,Humánerőforrás fejlesztés az Almási Balogh Pál...,ozd,2018-01-10 01:00:00,1.497179e+08,...,7.511878e+07,1.497179e+08,1.497179e+08,2020-06-05 01:00:00,2020-06-03 11:32:04,6.624433e+07,NaN,2020,2020-06-05 01:00:00,2020
18,1560420201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,bacs-kiskun megyei korhaz a szegedi tudomanyeg...,Egészségügyi humánerőforrás-fejlesztés,kecskemet,2017-12-19 01:00:00,1.487405e+08,...,0.000000e+00,1.487405e+08,1.487405e+08,2019-10-10 01:00:00,2018-11-27 11:10:01,1.461526e+08,NaN,2018,2019-10-10 01:00:00,2019
19,1604920201,Széchenyi 2020,EFOP,Egészségügyi humánerőforrás-fejlesztés,EFOP-1.10.2-17,bajai szent rokus korhaz,Bajai Szent Rókus Kórház humánerőforrás-fejles...,baja,2018-01-10 01:00:00,1.494150e+08,...,5.134631e+07,1.494150e+08,1.494150e+08,2019-06-14 01:00:00,2019-05-08 01:00:00,9.806869e+07,NaN,2019,2019-06-14 01:00:00,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,5881302,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,ricse nagykozseg onkormanyzata,"Egészségügyi központ nyílászáró cseréje, homlo...",ricse,2011-06-09 02:00:00,1.712726e+07,...,4.215049e+06,1.712726e+07,1.712726e+07,2013-06-20 01:00:00,2011-08-08 01:00:00,1.291221e+07,2013-06-09T23:00:00.000Z,2011,2013-06-20 01:00:00,2013
3963,5882702,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,sostofalva kozseg onkormanyzat,Orvosi rendelő felújítása,sostofalva,2011-06-09 02:00:00,1.269458e+07,...,0.000000e+00,1.269458e+07,1.269458e+07,2013-04-10 01:00:00,2011-08-05 01:00:00,1.262985e+07,2012-09-29T23:00:00.000Z,2011,2013-04-10 01:00:00,2013
3964,5879502,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,tiszababolna kozseg onkormanyzata,Orvosi rendelők és védőnői szolgálatok felújít...,tiszababolna,2011-06-09 02:00:00,2.614222e+07,...,1.521318e+06,2.614222e+07,2.614222e+07,2013-03-13 00:00:00,2011-10-06 01:00:00,2.461376e+07,2012-09-29T23:00:00.000Z,2011,2013-03-13 00:00:00,2013
3965,5882802,EU 2007-2013,ÉMOP,Egészségügyi szolgáltatások fejlesztése / Kis...,ÉMOP-4.1.1/C-10,tiszaladany kozsegi onkormanyzat,Orvosi rendelő bővítése és a védőnői szolgálat...,tiszaladany,2011-06-09 02:00:00,1.930096e+07,...,3.822521e+06,1.930096e+07,1.930096e+07,2013-06-07 01:00:00,2011-09-19 01:00:00,1.512913e+07,2012-12-30T23:00:00.000Z,2011,2013-06-07 01:00:00,2013


In [313]:
health_funds.to_csv('eu_health_funds_final.csv', encoding='utf-16')